In [1]:
#————————————————————

# Name: OpenAI Fine-Tuning(V1)
# Purpose: Create a fine-tuned openai GPT model that assist in the classification of product categories.
# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 10.01.2024
# Last Updated: 10.01.2024
# Python Version: 3.10.4

#General Sources:
#https://platform.openai.com/docs/api-reference?lang=python
#https://medium.com/ai-advances/complete-process-for-fine-tuning-gpt-3-5-turbo-using-openai-api-db4a50b3de1a

#Openai Usage:
#https://platform.openai.com/usage

#Additionals:
#The original data comes from #https://world.openfoodfacts.org/data
#The data used has been cleansed using the notebook named Data_Cleansing_V1


# Download Python packages (run the below command in terminal if packages have not yet been installed)
#pip install -r C:\Python\openai-lab\support\requirements\requirements.txt

#————————————————————

hello


In [3]:
# Import required libraries
import pandas as pd
import json
import numpy as np
import tiktoken
import time
import os
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import openai
from openai import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [4]:
# Import csv data

path_input = r"C:\Python\openai-lab\data\openfoodfacts.csv"
df = pd.read_csv(path_input , sep='\t')

In [5]:
# Create messages[{role :content}]: For role: system, user, assistant

training_data = []

system_message = "You are a helpful data categorizer. You are to extract the product categories from the provided products into three hierarchical product categories specified as Product Category 1, Product Category 2 and Product Category 3."

def create_user_message(row):
    return f"""Product: {row['product_name']}\n\nProduct Categories: """

def create_assistant_message(row):
    return f"""Product Category 1: {row['level_1']}, Product Category 2: {row['level_2']}, Product Category 3: {row['level_3']}"""

def create_final_message(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    assistant_message = create_assistant_message(row)
    messages.append({"role": "assistant", "content": assistant_message})

    return {"messages": messages}

create_final_message(df.iloc[0])

{'messages': [{'role': 'system',
   'content': 'You are a helpful data categorizer. You are to extract the product categories from the provided products into three hierarchical product categories specified as Product Category 1, Product Category 2 and Product Category 3.'},
  {'role': 'user',
   'content': 'Product: Apple Turnover\n\nProduct Categories: '},
  {'role': 'assistant',
   'content': 'Product Category 1: snacks, Product Category 2: sweet-snacks, Product Category 3: viennoiseries'}]}

In [6]:
# Split data into training and validation subsets

training_df = df.loc[0:100]
training_data = training_df.apply(create_final_message, axis=1).tolist()

validation_df = df.loc[101:150]
validation_data = validation_df.apply(create_final_message, axis=1).tolist()

test_df = df.loc[160:200]



In [7]:
# Convert to JSONL

def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

training_file_name = "training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [8]:
# Tokens count and cost estimation

# Load the jsonl dataset
with open(training_file_name, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("Example:")
for message in dataset[1]["messages"]:
    print(message)

Num examples: 101
Example:
{'role': 'system', 'content': 'You are a helpful data categorizer. You are to extract the product categories from the provided products into three hierarchical product categories specified as Product Category 1, Product Category 2 and Product Category 3.'}
{'role': 'user', 'content': 'Product: Bread\n\nProduct Categories: '}
{'role': 'assistant', 'content': 'Product Category 1: plant-based-foods-and-beverages, Product Category 2: plant-based-foods, Product Category 3: cereals-and-potatoes'}


In [9]:
# You can’t send more then 4096 tokens in each message, otherwise it will be truncated:

encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            # print(f'key={key}, value={value}, nbr_tokens={len(encoding.encode(value))}, cum_tokens={num_tokens}')
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

In [10]:
num_missing_system = 0
num_missing_user = 0
num_messages = []
message_all_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        num_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        num_missing_user += 1
    num_messages.append(len(messages))
    message_all_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num messages missing system message:", num_missing_system)
print("Num messages missing user message:", num_missing_user)
print("\nSome statistics:")
print(f"min={min(message_all_lens)}, max={max(message_all_lens)}")
print(f"p5={np.quantile(message_all_lens, 0.1)}, p95={np.quantile(message_all_lens, 0.95)}")

print("\Examples of tokens nbr from the first messages")
print(message_all_lens[:5])

num_sample_token_long = sum(l > 4096 for l in message_all_lens)

if num_sample_token_long>0:
  print(f"\n{num_sample_token_long} messages may be over the 4096 token limit, they will be truncated during fine-tuning")
else:
  print(f"\nNo message will be truncated during fine-tuning")

Num messages missing system message: 0
Num messages missing user message: 0

Some statistics:
min=86, max=110
p5=91.0, p95=105.0
\Examples of tokens nbr from the first messages
[92, 98, 92, 97, 100]

No message will be truncated during fine-tuning


In [11]:
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 8
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in message_all_lens)
print(f"Dataset contains ~{n_tokens_in_dataset} tokens, which will incur training charges")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_tokens_in_dataset} tokens")
print(f"The final cost will be ~{n_epochs * n_tokens_in_dataset/1000  * 0.008}$")

Dataset contains ~9790 tokens, which will incur training charges
By default, you'll train for 8 epochs on this dataset
By default, you'll be charged for ~78320 tokens
The final cost will be ~0.62656$


In [12]:
# Load OpenAIKey
load_dotenv(dotenv_path=Path("C:\Python\openai-lab\.venv\.env"))
API_KEY = os.environ['OPENAI_API_KEY']

# Initalise Large Language Model (LLM)

llm = OpenAI(api_key = API_KEY)

In [13]:
# Upload JSONL files to OpenAI

training_file_creation = llm.files.create(
  file=open(training_file_name, "rb"),
  purpose='fine-tune'
)


id_file_training = training_file_creation.id

validation_file_creation = llm.files.create(
  file=open(validation_file_name, "rb"),
  purpose='fine-tune'
)

id_file_validation = validation_file_creation.id

In [14]:
# Create Fine-Tuned Model

fine_tuning_job = llm.fine_tuning.jobs.create(training_file=id_file_training,
                            model="gpt-3.5-turbo",
                            suffix='categorizer',
                            hyperparameters = {
                                'n_epochs':8
                            },
                            validation_file=id_file_validation)

fine_tuning_job

FineTuningJob(id='ftjob-5kZRJumX73i2p12u7u9z46I6', created_at=1704885146, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=8, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-D8U9yUgZdB4bBF6lEOriShz7', result_files=[], status='validating_files', trained_tokens=None, training_file='file-c63YYtmVcjhgTj3htqpzZfsY', validation_file='file-a6uYX8du5h6uoNZLkfeKH35U')

In [16]:
#Retrieve Model Status
print("checking status. ")
while True:
    run = llm.fine_tuning.jobs.retrieve(fine_tuning_job.id)

    if run.status == "succeeded":
        print("finished!")
        messages = run

        break

    else:
        events_list_job = llm.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tuning_job.id, limit=50)
        events = events_list_job.data
        events.reverse()

        for event in events:
            print(event.message)

    time.sleep(40)
print (messages)

checking status. 
finished!
FineTuningJob(id='ftjob-5kZRJumX73i2p12u7u9z46I6', created_at=1704885146, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:allgeier-schweiz-ag:categorizer:8fRCWcue', finished_at=1704886699, hyperparameters=Hyperparameters(n_epochs=8, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-D8U9yUgZdB4bBF6lEOriShz7', result_files=['file-bdYjLLzQ2hKvi4adeVfa9cJW'], status='succeeded', trained_tokens=76704, training_file='file-c63YYtmVcjhgTj3htqpzZfsY', validation_file='file-a6uYX8du5h6uoNZLkfeKH35U')


In [17]:
retrieve_fine_tuned_job = llm.fine_tuning.jobs.retrieve(fine_tuning_job_id=fine_tuning_job.id)
fine_tuned_model_id = retrieve_fine_tuned_job.fine_tuned_model

In [ ]:
# Test Fine-Tuned model with one input

new_input = "Nutella"

def create_user_message():
    return f"""Product: {new_input}\n\nProduct Categories: """

def create_final_message_test():
  test_messages = []
  test_messages.append({"role": "system", "content": system_message})

  user_message = create_user_message()
  test_messages.append({"role": "user", "content": create_user_message()})

  return test_messages

messages_test = create_final_message_test()

response = llm.chat.completions.create(
    model=fine_tuned_model_id,
    messages=messages_test
  )

response.choices[0].message.content

In [18]:
# Test Fine-Tuned model with one test_dataset input for accuracy testing


def create_final_message_test(row):
  test_messages = []
  test_messages.append({"role": "system", "content": system_message})

  user_message = create_user_message(row)
  test_messages.append({"role": "user", "content": create_user_message(row)})

  return test_messages

finetuned_model_resp =[]

for i in range(0, len(test_df)):
  row = test_df.iloc[i]
  messages_test = create_final_message_test(row)

  response = llm.chat.completions.create(
    model=fine_tuned_model_id,
    messages=messages_test
  )

  finetuned_model_resp.append(response.choices[0].message.content)

In [ ]:
test_df['result'] = finetuned_model_resp

In [21]:
y_true = test_df['result'].values
accuracy = accuracy_score(y_true, finetuned_model_resp)
precision = precision_score(y_true, finetuned_model_resp, average=None)
recall = recall_score(y_true, finetuned_model_resp,average=None)
f1 = f1_score(y_true, finetuned_model_resp,average=None)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 1.0
Precision: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Recall: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 Score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
